In [1]:
import ipywidgets as widgets
import pandas as pd
from github import Github
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import numpy as np

In [2]:
reset = True

check = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)


    

def show_players(check):

    response = requests.get("https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/main/2022_2023/data.csv")
    soup = BeautifulSoup(response.content, "html.parser")
    
    hash_last_commit = soup.find("a", {"class": "d-none js-permalink-shortcut"}).get("href").split("/")[-3]
    
    url_data = fr"https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/{hash_last_commit}/2022_2023/data.csv"
    data_csv = pd.read_csv(url_data, header=None)
    
    fig = plt.figure()
    counter_row = 0
    for index, row in data_csv.iterrows():
        for counter, feat in enumerate(row):
            if counter == 1:
                plt.text(counter, counter_row, float(feat) + np.random.random())
            else:
                plt.text(counter, counter_row, feat)
        counter_row += 1

    plt.xlim(0, 5)
    plt.ylim(0, len(data_csv) + 1)
    plt.title(np.random.random())
    plt.show()
    
    reset = False
    
def on_button_clicked(_):
    reset = True
    check.value = not check.value

button = widgets.Button(description='Update player list')
button.on_click(on_button_clicked)

display(widgets.VBox([button]))
widgets.interactive_output(show_players, {"check": check})

Output()

In [3]:
reset = True

check__2 = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)


    

def show_players__2(check):

    response = requests.get("https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/main/2022_2023/data.csv")
    soup = BeautifulSoup(response.content, "html.parser")
    
    hash_last_commit = soup.find("a", {"class": "d-none js-permalink-shortcut"}).get("href").split("/")[-3]
    
    url_data = fr"https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/{hash_last_commit}/2022_2023/data.csv"
    data_csv = pd.read_csv(url_data, header=None)
    
    fig__2 = plt.figure()
    counter_row = 0
    for index, row in data_csv.iterrows():
        for counter, feat in enumerate(row):
            if counter == 1:
                plt.text(counter, counter_row, float(feat) + np.random.random())
            else:
                plt.text(counter, counter_row, feat)
        counter_row += 1

    plt.xlim(0, 5)
    plt.ylim(0, len(data_csv) + 1)
    plt.title(np.random.random())
    plt.show()
    
    reset = False
    
def on_button_clicked__2(_):
    reset = True
    check__2.value = not check__2.value

button__2 = widgets.Button(description='Update player list')
button__2.on_click(on_button_clicked__2)

display(widgets.VBox([button__2]))
widgets.interactive_output(show_players__2, {"check": check__2})

Output()

In [ ]:
reset = True

check__3 = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)


    

def show_players__3(check):

    response = requests.get("https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/main/2022_2023/data.csv")
    soup = BeautifulSoup(response.content, "html.parser")
    
    hash_last_commit = soup.find("a", {"class": "d-none js-permalink-shortcut"}).get("href").split("/")[-3]
    
    url_data = fr"https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/{hash_last_commit}/2022_2023/data.csv"
    data_csv = pd.read_csv(url_data, header=None)
    
    fig__3 = plt.figure()
    counter_row = 0
    for index, row in data_csv.iterrows():
        for counter, feat in enumerate(row):
            if counter == 1:
                plt.text(counter, counter_row, float(feat) + np.random.random())
            else:
                plt.text(counter, counter_row, feat)
        counter_row += 1

    plt.xlim(0, 5)
    plt.ylim(0, len(data_csv) + 1)
    plt.title(np.random.random())
    plt.show()
    
    reset = False
    
def on_button_clicked__3(_):
    reset = True
    check__3.value = not check__3.value

button__3 = widgets.Button(description='Update player list')
button__3.on_click(on_button_clicked__3)

display(widgets.VBox([button__3]))
widgets.interactive_output(show_players__3, {"check": check__3})

In [ ]:
reset = True

check__4 = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)


    

def show_players__4(check):

    response = requests.get("https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/main/2022_2023/data.csv")
    soup = BeautifulSoup(response.content, "html.parser")
    
    hash_last_commit = soup.find("a", {"class": "d-none js-permalink-shortcut"}).get("href").split("/")[-3]
    
    url_data = fr"https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/{hash_last_commit}/2022_2023/data.csv"
    data_csv = pd.read_csv(url_data, header=None)
    
    fig__4 = plt.figure()
    counter_row = 0
    for index, row in data_csv.iterrows():
        for counter, feat in enumerate(row):
            if counter == 1:
                plt.text(counter, counter_row, float(feat) + np.random.random())
            else:
                plt.text(counter, counter_row, feat)
        counter_row += 1

    plt.xlim(0, 5)
    plt.ylim(0, len(data_csv) + 1)
    plt.title(np.random.random())
    plt.show()
    
    reset = False
    
def on_button_clicked__4(_):
    reset = True
    check__4.value = not check__4.value

button__4 = widgets.Button(description='Update player list')
button__4.on_click(on_button_clicked__4)

display(widgets.VBox([button__4]))
widgets.interactive_output(show_players__4, {"check": check__4})